<a href="https://colab.research.google.com/github/alessioborgi/DL_Project/blob/main/Source/InfoRetrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

==================================================

**Project Name:** Neural inverted index for fast and effective information retrieval\
**Course:** Deep Learning\
**University:** Sapienza Università di Roma

**Authors:**
  - [Alessio Borgi] (<tt>1952442</tt>)
  - [Eugenio Bugli] (<tt>1934824</tt>)
  - [Damiano Imola] (<tt>2109063</tt>)

**Date:** [November 2024 - Completion Date]

==================================================

# 0: INSTALL & IMPORT LIBRARIES

In [2]:
!pip install -q --upgrade pip
# !pip install -q pyserini==0.12.0
!pip install -q pyserini==0.21.0
!pip install -q faiss-gpu
!pip install -q pytorch-lightning transformers datasets torch wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 145.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 44.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

In [4]:
# base
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from collections import Counter
from typing import List, Tuple

# could plots
import wandb

# torch
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

# lightning
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor, StochasticWeightAveraging, DeviceStatsMonitor

# HF and similar
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from transformers import AutoModel, AutoTokenizer, AutoTokenizer, AutoModelForSequenceClassification, T5Tokenizer, T5ForConditionalGeneration

# sklearn
from sklearn.preprocessing import normalize
from sklearn.cluster import AgglomerativeClustering, KMeans

# pyserini
import faiss
from pyserini.index import IndexReader
from pyserini.search import SimpleSearcher
from pyserini.search.lucene import LuceneSearcher

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
wandb.login()
# wandb.login(key="b3bce19a09c51bdf8a19eb3dc58f7c44de929e13") #(ALESSIO)
# wandb.login(key="6d550e12a1b8f716ebe580082f495c01ed2adf6c") #(DAMIANO)
wandb.init(project="IR_DSI", resume="allow")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: alessioborgi3 (adavit). Use `wandb login --relogin` to force relogin


# 1: DOWNLOADING DATASET




In [ ]:
# PyTorch Dataset class
class MSMARCODataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128, is_test=False):
        """
        Initialize the dataset for MS MARCO.

        Args:
            data: The dataset split (train, validation, or test).
            tokenizer: The tokenizer instance.
            max_length: Maximum token length for inputs.
            is_test: Flag to indicate if the dataset is a test set (no labels).
        """
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.is_test = is_test

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        query = item["query"]
        # COMMENT (D): bisogna ovviamente cambiarlo
        # soprattutto perchè loro utilizzano anche uno skip dei primi K chunk di un documento per controllare
        # quanto il modello overfitta sui primi chunks (e quindi quando riesce a catturare la semantica del DOCUMENTO INTERO)
        passage = item["passages"]["passage_text"][0]

        # If not test set, fetch the label
        # COMMENT (D): non necessario, grazie al controllo finale
        # label = None if self.is_test else 1 if item["passages"]["is_selected"][0] else 0

        # Tokenize input
        # COMMENT (D): dobbiamo controllare se l'impostazione del tokenizer è corretta
        inputs = self.tokenizer(
            query,
            passage,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        )

        result = {
            "input_ids": inputs["input_ids"].squeeze(0),
            # COMMENT (D): ci serve anche l'attention mask?
            "attention_mask": inputs["attention_mask"].squeeze(0)
        }

        # Add label only if it's not the test set
        # if not self.is_test:
        #     result["label"] = torch.tensor(label, dtype=torch.long)

        return result

In [ ]:
class MSMarcoDataModule(pl.LightningDataModule):
    def __init__(self, train_data, validation_data, test_data, tokenizer, batch_size=32):
        """
        Data module for handling MS MARCO datasets.

        Args:
            train_data: Training dataset split.
            validation_data: Validation dataset split.
            test_data: Test dataset split.
            tokenizer: The tokenizer instance.
            batch_size: Batch size for data loaders.
        """
        super().__init__()
        self.train_data = train_data
        self.validation_data = validation_data
        self.test_data = test_data
        self.tokenizer = tokenizer
        self.batch_size = batch_size

    def setup(self, stage=None):
        self.train_dataset = MSMARCODataset(self.train_data, self.tokenizer)
        self.val_dataset = MSMARCODataset(self.validation_data, self.tokenizer)
        self.test_dataset = MSMARCODataset(self.test_data, self.tokenizer, is_test=True)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size)

In [ ]:
# Load MS MARCO splits
ms_marco_train = load_dataset("microsoft/ms_marco", "v1.1", split="train")
ms_marco_validation = load_dataset("microsoft/ms_marco", "v1.1", split="validation")
ms_marco_test = load_dataset("microsoft/ms_marco", "v1.1", split="test")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.48k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/175M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/10047 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/82326 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9650 [00:00<?, ? examples/s]

# 2: DATASET EXPLORATION

In [ ]:
# Function to print dataset characteristics.
def print_dataset_info(name, dataset):
    print(f"\nDataset: {name}")
    print("-" * 40)
    print(f"Number of samples: {len(dataset)}")
    print(f"Features: {dataset.features.keys()}")
    print("\nExample:")
    for k in dataset.features.keys():
        print('\t', f'{k}: ', dataset[0][k])
        if(k == 'passages'):
            print('\t\t', "Number of passages:", len(dataset[0][k]['passage_text']))
            for i in range(len(dataset[0][k]['passage_text'])):
                print('\t\t', f'Passage {i}: ', dataset[0][k]['passage_text'][i])
    print('\n\n')

# Print information for each split.
print_dataset_info("Train", ms_marco_train)
print_dataset_info("Validation", ms_marco_validation)
print_dataset_info("Test", ms_marco_test)


Dataset: Train
----------------------------------------
Number of samples: 82326
Features: dict_keys(['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'])

Example:
	 answers:  ['Results-Based Accountability is a disciplined way of thinking and taking action that communities can use to improve the lives of children, youth, families, adults and the community as a whole.']
	 passages:  {'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0], 'passage_text': ["Since 2007, the RBA's outstanding reputation has been affected by the 'Securency' or NPA scandal. These RBA subsidiaries were involved in bribing overseas officials so that Australia might win lucrative note-printing contracts. The assets of the bank include the gold and foreign exchange reserves of Australia, which is estimated to have a net worth of A$101 billion. Nearly 94% of the RBA's employees work at its headquarters in Sydney, New South Wales and at the Business Resumption Site.", "The Reserve Bank of Aust

In [ ]:
# Analyze specific features.
def analyze_passages(dataset):
    print("\n--- Passage Analysis ---")
    passage_lengths = [len(p["passage_text"][0]) for p in dataset["passages"]]
    print(f"Number of passages per query: {len(dataset[0]['passages']['passage_text'])}")
    print(f"Average passage length: {sum(passage_lengths) / len(passage_lengths):.2f} characters")
    print(f"Max passage length: {max(passage_lengths)} characters")
    print(f"Min passage length: {min(passage_lengths)} characters")

# Analyze passages in the train set.
analyze_passages(ms_marco_train)


--- Passage Analysis ---
Number of passages per query: 10
Average passage length: 414.24 characters
Max passage length: 1167 characters
Min passage length: 43 characters


In [ ]:
# Initialize tokenizer.
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Create data module
data_module = MSMarcoDataModule(
    train_data=ms_marco_train,
    validation_data=ms_marco_validation,
    test_data=ms_marco_test,
    tokenizer=tokenizer,
    batch_size=32
)

# Prepare datasets
data_module.setup()

# Access dataloaders
train_loader = data_module.train_dataloader()
val_loader = data_module.val_dataloader()
test_loader = data_module.test_dataloader()

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
# Iterate through the training loader
for batch in train_loader:
    input_ids = batch["input_ids"]  # Tokenized input IDs
    attention_mask = batch["attention_mask"]  # Attention mask
    print("Batch input_ids shape:", input_ids.shape)
    print("Batch attention_mask shape:", attention_mask.shape)
    print("Batch labels shape:", labels.shape)
    break  # Stop after printing one batch

# 3: DATASET GENERATION

The core idea is that the model must create associations between `queries` and `docids`.

<h2>What do we have to do?</h2>

As the (second) paper mention, we need to construct a dataset $\mathcal{T}'$.\
To do so, we start from the generation of $$\mathcal{U} = \mathcal{O} \cup \mathcal{P}\qquad
\begin{cases}
\mathcal{O} = \bigcup_i \mathcal{O}_i = \bigcup_i \{d^1_i, d^2_i, \dots, d^m_i\}\\[10pt]
\mathcal{P} = \bigcup_i \mathcal{P}_i = \bigcup_i \{pq^1_i, pq^2_i, \dots, pq^m_i\}
\end{cases}$$
with
*   $d_i^j$ the $j$-th segment of the $i$-th document that belongs to the set $\mathcal{D}$ and
*   $pq_i^j$ the pseudo-query generated by [docT5query](https://github.com/castorini/docTTTTTquery) against the document segment $d_i^j$

then, having such computed $\mathcal{U}$, we need to filter it using a so called *'dense model'* named $M$. Due to its nature, dense retrieval models effectively preserve textual information in its representation (i.e. in its latent space), so they are perfect to address such task.

> We need to find a dense retrieval model open-source



<h2>How to filter?</h2>

We need to input individual fragment $t$ originating from the document of $id_t$ that belongs to $U$ in our dense retrieval model $M$; so our $t$ now behaves like a query
$$t\in d_t = doc(id_t)\in U$$
this process outputs a ranked list of $k$ elements
$$R_k(t, M) = (id^1, id^2,\dots, id^k)$$
if $id_t$ belongs to $R_k(t, M)$ this means that *'the fragment $t$ possessed keyinformation relevant to the original document'* and so can be included in the training corpus $T'$.\
In other words, in this way we asses that $t$ har enough information to represent the document identified by $id_t$ ans so it is a suitable query.

We'll proceed by steps, applying the real preprocessing we mentioned earlied on the vanilla dataset created; hence, in order:

1.   semantically structured docids as explained in the original paper (i.e. using KMeans)
2.   pseudo-query generation using [docT5query](https://github.com/castorini/docTTTTTquery)
3.   tokenization of document' segments using the sentence encoder [all-MiniLM-L6-v2](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2)




## 3.1: SEMANTICALLY STRUCTURED DOCIDS CREATION




In [ ]:
def generate_semantic_ids(document_embeddings, n_clusters = 10, max_docs_per_cluster = 100, depth = 0):
    n_docs, _ = document_embeddings.shape
    cluster_prefix = []

    # base case, whether we have a cluster with at most
    # 'c' documents, we return their index in current cluster
    if n_docs <= max_docs_per_cluster:
        # return [f"{depth}-{i}" for i in range(n_docs)]
        return [f"{i}" for i in range(n_docs)]

    # k-means clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(document_embeddings)

    cluster_labels = kmeans.labels_
    centroids = kmeans.cluster_centers_

    # generate docids for each cluster
    structured_ids = []
    for cluster_id in range(n_clusters):
        # mask for documents of the current centroid
        cluster_indices = np.where(cluster_labels == cluster_id)[0]
        cluster_embeddings = document_embeddings[cluster_indices]

        # doids for the sub-cluster
        sub_ids = generate_semantic_ids(cluster_embeddings, n_clusters, max_docs_per_cluster, depth + 1)

        # cluster_prefix = [f"{depth}-{cluster_id}"] * len(sub_ids)
        cluster_prefix = [f"{cluster_id}"] * len(sub_ids)

        # combine parent prefix with sub-cluster identifiers
        # structured_ids.extend([f"{prefix}-{sub_id}" for prefix, sub_id in zip(cluster_prefix, sub_ids)])
        structured_ids.extend([f"{prefix}{sub_id}" for prefix, sub_id in zip(cluster_prefix, sub_ids)])

    return structured_ids

In [ ]:
def generate_semantically_structured_docids(dataset, sentence_embedder, n_clusters=10, max_docs_per_cluster=10, start_from=0, stop_at=100):
    document_corpus = []
    document_embeddings = []

    # iterate within the range start_from to stop_at
    for i in range(start_from, min(len(dataset), stop_at + 1)):
        if i % 10 == 0: print("Processing document: ", i)

        # retrieve all passages in a document
        passages = dataset[i]['passages']['passage_text']

        # add the entire corpus of a document
        document_corpus.append(' '.join(passages))

        # compute embeddings for each passage
        passages_embeddings = sentence_embedder.encode(passages)

        # average passages embeddings to maintain semantic meaning
        document_embedding = np.mean(passages_embeddings, axis=0)

        # append one document embedding
        document_embeddings.append(document_embedding)

    ssdocids = generate_semantic_ids(np.array(document_embeddings), n_clusters=n_clusters, max_docs_per_cluster=max_docs_per_cluster)

    return document_embeddings, document_corpus, ssdocids

In [ ]:
sentence_embedder = SentenceTransformer('all-MiniLM-L6-v2')

start_from = 0
stop_at = 100

document_embeddings, document_corpus, ssdocids = generate_semantically_structured_docids(
    ms_marco_train, sentence_embedder, n_clusters=10, max_docs_per_cluster=10, start_from=start_from, stop_at=stop_at)

# output_file = f"semantically_structured_docids_train_{start_from}_{stop_at}.json"

# with open(output_file, "w") as f:
#     json.dump(ssdocids, f, indent=4)

# print(f"Semantically structured document IDs saved to {output_file}")

Processing document:  0
Processing document:  10
Processing document:  20
Processing document:  30
Processing document:  40
Processing document:  50
Processing document:  60
Processing document:  70
Processing document:  80
Processing document:  90
Processing document:  100


## 3.2: DOCUMENT SPLITTING AND PSEUDO QUERIES (DATASETS $O$ AND $P$)

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('castorini/doc2query-t5-base-msmarco')
model = T5ForConditionalGeneration.from_pretrained('castorini/doc2query-t5-base-msmarco').to(device)

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [ ]:
def generate_pseudo_queries(dataset, tokenizer, retrieval_model, max_pseudo_query_len=64, top_k=1, start_from=0, stop_at=100, debug=False):
    pseudo_queries = [] # P
    passages = [] # O

    for i in range(start_from, min(len(dataset), stop_at + 1)):
        if i % 10 == 0: print("Processing document: ", i)

        l = len(dataset[i]['passages']['passage_text'])

        for j in range(l):

            t = dataset[i]['passages']['passage_text'][j]
            passages.append(t)

            # tokenize the segment (i.e. the passage)
            input_ids = tokenizer.encode(t, return_tensors='pt').to(device)

            # doct5query predict top k(=1) queries
            outputs = model.generate(
                input_ids=input_ids,
                max_length=max_pseudo_query_len,
                do_sample=True,
                top_k=top_k,
                num_return_sequences=1
            ).to(device)

            # decode the (pseudo-)query
            decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

            if debug:
                print(f'SAMPLE {j + 1}: \n passage: {t}\n pseudo-query: {decoded}')
                print('-'*50)

            pseudo_queries.append({'document_index': i, "pseudo-query": decoded})

    return passages, pseudo_queries

In [ ]:
top_k = 1
max_pseudo_query_len = 64
start_from = 0
stop_at = 100
debug = True

passages, pseudo_querries = generate_pseudo_queries(ms_marco_train, tokenizer, model, max_pseudo_query_len, top_k, start_from, stop_at, debug)

## 3.3 GENERATE DATASET $U$

In [ ]:
sentence_embedder = SentenceTransformer('all-MiniLM-L6-v2')

tokenizer = T5Tokenizer.from_pretrained('castorini/doc2query-t5-base-msmarco')
model = T5ForConditionalGeneration.from_pretrained('castorini/doc2query-t5-base-msmarco').to(device)

start_from = 0
stop_at = 100

max_pseudo_query_lenght = 64
top_k = 1

In [ ]:
# embeddings and docids for each document
document_embeddings, document_corpus, ssdocids = generate_semantically_structured_docids(
    ms_marco_train,
    sentence_embedder,
    n_clusters = 10,
    max_docs_per_cluster = 10,
    start_from = start_from,
    stop_at = stop_at)

# pseudo_queries for each sentence
pseudo_queries = generate_pseudo_queries(
    ms_marco_train,
    tokenizer = tokenizer,
    retrieval_model = model,
    max_pseudo_query_len = max_pseudo_query_lenght,
    top_k = top_k,
    start_from = start_from,
    stop_at = stop_at,
    debug=False)

In [ ]:
len(pseudo_queries), len(ssdocids)

(823, 101)

$$\mathcal{U}= \mathcal{O}\cup \mathcal{P}$$

In [ ]:
def generate_samples_to_rank(dataset, ssdocids, pseudo_queries, start_from=0, stop_at=100):
    resulting_dataset = []

    for i in range(start_from, min(len(dataset), stop_at + 1)):
        l = len(dataset[i]['passages']['passage_text'])

        docid = ssdocids[i]

        for j in range(l):
            resulting_dataset.append({
                'doc_id': docid,
                'doc_segment': dataset[i]['passages']['passage_text'][j],
                'segment_pseudo_query': pseudo_queries[j]['pseudo-query']})

    return resulting_dataset

In [ ]:
U = generate_samples_to_rank(ms_marco_train, ssdocids, pseudo_queries)

with open("U.json", "w") as f:
    json.dump(U, f, indent=4)

## 3.4: FILTERING: RETRIEVAL MODELS FOR RANKING

We are going to use the MSMARCO-100k, so we'll apply each of these stuffs on it.

1.   For the pseudo-queries generation, we can use the [docT5query](https://github.com/castorini/docTTTTTquery) from castorini
2.   As a *'dense retrieval model'* for ranking documents we can use the one proposed by castorini: [FAISS](https://github.com/castorini/pyserini/blob/master/docs/usage-search.md#faiss)
3.   For the segments extraction we can rely on a manually crafted approach

In order to better check the improvements lead from a dense retrieval model w.r.t. a sparse one, we'll use an approach based also on Lucene (a sparse retrieval model) and an hybrid one, combining both FAISS and Lucene.


The hybrid retrieval pipeline we propose combines **dense retrieval** (using FAISS) and **sparse retrieval** (using Lucene with BM25). The objective of this step is to rank documents based on both semantic similarity and lexical matching to achieve robust results.

---
As a first, step, we need to use a `SentenceTransformer` model to encode documents into dense vector embeddings, which are then indexed by FAISS.

- For a document $d_i$, the embedding is represented as:
  $$
  \mathbf{e}_{d_i} = f_{\text{dense}}(d_i)
  $$
  where $f_{\text{dense}}$ is the embedding function provided by the `SentenceTransformer` model.

- The embeddings of all documents are stored in a matrix:
  $$
  \mathbf{E} = [\mathbf{e}_{d_1}, \mathbf{e}_{d_2}, \dots, \mathbf{e}_{d_N}] \in \mathbb{R}^{N \times D}
  $$
  Here, $N$ is the number of documents, and $D$ is the embedding dimensionality.

Similarly, the query $q$ is encoded into a dense vector:
$$
\mathbf{e}_q = f_{\text{dense}}(q)
$$

Assuming we have a temporary dataset consisting of `(doc_id, doc_segment, segment_pseudo_query)`\
I'll sketch a pseudocode (since we need to build such temporary dataset)

In [ ]:
def build_dataset_with_ranker(dataset, rank, index, sentence_embedder, top_k=1):
    final_dataset = []

    for elem in dataset:
        doc_id = elem['doc_id']
        t = elem['doc_segment']
        pq = elem['segment_pseudo_query']

        # lucene
        if sentence_embedder is None:
            distances, indices = rank(t, index, top_k=1)
        # faiss
        else:
            query_embedding = sentence_embedder.encode([t], convert_to_numpy=True)
            distances, indices = rank(query_embedding, index, top_k=1)

        # FIXME: check if indices are actually doc_ids
        if doc_id in indices:
            final_dataset.append({
                'docid': doc_id,
                'segment': t,
                'pseudo-query': pq,
                'ranker_distance': distances.indexof(doc_id)
            })

    return final_dataset

### 3.4.1: FAISS DENSE RETRIEVAL MODEL






- **Index Creation**: The normalized document embeddings are added to a FAISS `IndexFlatIP` index.
- **Querying**: Given the query embedding $\mathbf{e}_q$, FAISS retrieves the top-$k$ documents based on inner product:
$$
\text{score}_{\text{FAISS}}(q, d_i) = \mathbf{e}_q \cdot \mathbf{e}_{d_i}
$$

The FAISS search returns:
- `indices`: The IDs of the top-$k$ documents.
- `distances`: The corresponding similarity scores.

<h3> NORMALIZATION FOR COSINE SIMILARITY </h3>

FAISS uses **inner product** for similarity scoring by default. To ensure the scores are equivalent to cosine similarity, we apply a simple trick consisting in making both the document embeddings and the query embedding to be normalized:
$$
\mathbf{e}' = \frac{\mathbf{e}}{\|\mathbf{e}\|}
$$
Where $\|\mathbf{e}\|$ is the L2 norm of the embedding.

Normalized embeddings allow the inner product to behave like cosine similarity:
$$
\text{cosine_similarity}(\mathbf{e}_q, \mathbf{e}_{d_i}) = \mathbf{e}_q \cdot \mathbf{e}_{d_i}
$$

In [ ]:
document_embeddings = np.array(document_embeddings)

# latent (embedding) dim
dim = document_embeddings.shape[1]

# based on cosine similarity (IP = Inner Product)
faiss_index = faiss.IndexFlatIP(dim)

# normalize and add document embeddings to FAISS index
faiss.normalize_L2(document_embeddings)
faiss_index.add(document_embeddings)

In [ ]:
def ranker_faiss(query_embedding, faiss_index, top_k=1):
    faiss.normalize_L2(query_embedding)

    # Perform FAISS search
    faiss_distances, faiss_indices = faiss_index.search(query_embedding, top_k)

    return faiss_distances, faiss_indices

In [ ]:
faiss_dataset = build_dataset_with_ranker(U, ranker_faiss, faiss_index, sentence_embedder)

### 3.4.2: LUCENE SPARSE RETRIEVAL MODEL


Lucene uses the **BM25** algorithm for sparse retrieval. For a query $q$ and document $d_i$, the BM25 relevance score is given by:
$$
\text{BM25}(q, d_i) = \sum_{t \in q} \text{IDF}(t) \cdot \frac{f(t, d_i) \cdot (k_1 + 1)}{f(t, d_i) + k_1 \cdot (1 - b + b \cdot \frac{|d_i|}{\text{avgdl}})}
$$
Where:
- $t$: A term in the query.
- $f(t, d_i)$: The frequency of term $t$ in document $d_i$.
- $|d_i|$: The length of the document.
- $\text{avgdl}$: The average document length in the corpus.
- $k_1$: Controls term frequency saturation (default $1.2$).
- $b$: Controls length normalization (default $0.75$).
- $\text{IDF}(t)$: The inverse document frequency of term $t$:
  $$
  \text{IDF}(t) = \log\left(\frac{N - n_t + 0.5}{n_t + 0.5} + 1\right)
  $$
  Where $N$ is the total number of documents and $n_t$ is the number of documents containing term $t$.




Pyserini retrieves the top-$k$ documents using BM25 scoring. The results include:
- `docid`: Document IDs.
- `BM25_score`: BM25 relevance scores.



In [ ]:
jsonl_path = "documents.jsonl"
index_path = "lucene-index"

# pairs (doc_id, doc_text) written into a JSONL file
with open(jsonl_path, "w") as f:
    for doc_id, content in zip(ssdocids, document_corpus):
        f.write(json.dumps({"id": doc_id, "contents": content}) + "\n")

# clear previous index (if any)
os.system(f"rm -rf {index_path}")

# build lucene index
os.system(f"python -m pyserini.index -collection JsonCollection "
          f"-generator DefaultLuceneDocumentGenerator "
          f"-threads 1 -input . -index {index_path} -storeRaw")

# rank segments
lucene_index = LuceneSearcher(index_path)
lucene_index.set_bm25(k1=0.9, b=0.4)

In [ ]:
def ranker_lucene(pseudo_query, lucene_index, top_k=1):
    lucene_hits = lucene_index.search(pseudo_query, k=top_k)

    lucene_distances = []
    lucene_indices = []

    for rank, hit in enumerate(lucene_hits):
        lucene_indices.append(hit.docid)
        lucene_distances.append(hit.score)

    return lucene_distances, lucene_indices

In [ ]:
lucene_dataset = build_dataset_with_ranker(U, ranker_lucene, lucene_index, sentence_embedder)

### 3.4.3: HYBRID RETRIEVAL MODEL


To leverage the strengths of both retrieval approaches, we combine the scores from FAISS and Lucene.

The combined score for each document is calculated through a **Linear Score Fusion**, i.e., as a weighted sum:
$$
\text{Combined_Score}(q, d_i) = \alpha \cdot \text{score}_{\text{FAISS}}(q, d_i) + (1 - \alpha) \cdot \text{BM25}(q, d_i)
$$
Where:
- $\alpha \in [0, 1]$: Weight parameter controlling the contribution of FAISS vs. BM25 scores.

1. **Semantic Understanding**:
   - FAISS captures semantic relationships between queries and documents.
   - Example: "What is deep learning?" matches "Neural networks power deep learning."

2. **Lexical Precision**:
   - BM25 ensures term-matching precision.
   - Example: "What is deep learning?" prefers documents explicitly mentioning "deep learning."

3. **Robustness**:
   - The hybrid approach balances semantic and lexical relevance, reducing the risk of missing relevant documents.

In [ ]:
alpha = 0.1
weight_dense = False
normalization = True

In [ ]:
dense_hits = {hit['segment']: hit['ranker_distances'] for hit in faiss_dataset}
sparse_hits = {hit['segment']: hit['ranker_distances'] for hit in lucene_dataset}

hybrid_result = []

# retrieve the minimum and maximum score obtained by both dense
# and sparse indices for computing mean normalization
min_dense_score = min(dense_hits.values()) if len(dense_hits) > 0 else 0
max_dense_score = max(dense_hits.values()) if len(dense_hits) > 0 else 1
min_sparse_score = min(sparse_hits.values()) if len(sparse_hits) > 0 else 0
max_sparse_score = max(sparse_hits.values()) if len(sparse_hits) > 0 else 1

for doc in set(dense_hits.keys()) | set(sparse_hits.keys()):
    # document only in sparse index
    if doc not in dense_hits:
        sparse_score = sparse_hits[doc]
        dense_score = min_dense_score
    # document only in dense index
    elif doc not in sparse_hits:
        sparse_score = min_sparse_score
        dense_score = dense_hits[doc]
    # document in both indices
    else:
        sparse_score = sparse_hits[doc]
        dense_score = dense_hits[doc]

    # mean normalization of scores
    if normalization:
        sparse_score = (sparse_score - (min_sparse_score + max_sparse_score) / 2) / (max_sparse_score - min_sparse_score)
        dense_score = (dense_score - (min_dense_score + max_dense_score) / 2) / (max_dense_score - min_dense_score)

    # compute the weighted score
    if weight_dense:
        score = alpha * sparse_score + (1 - alpha) * dense_score
    else:
        score = (1 - alpha) * sparse_score + alpha * dense_score

    hybrid_result.append(DenseSearchResult(doc, score))

#### 3.4.3.1: Hierarchical Navigable Small Worlds (HNSW)


Due to the big amount of document we need to process, we opt for faster approximat nearest neighbor search exploited by FAISS.\
Since the original FAISS implementation uses the brute force approach, parsing a huge quantity of document can result in a prohibitive processing time; the addendum lead by approximat nearest neighbor search allows us to speed-up such operation without loosing to much in performances.

### 3.4.4 SUPERVISION DISTILLATION SIGNAL

## 4: MODEL

In [7]:
import torch
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from transformers import T5Tokenizer, T5ForConditionalGeneration, EncoderDecoderCache
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.cluster import AgglomerativeClustering

# Define the model class
class DSIT5Model(pl.LightningModule):
    def __init__(self, model_name="t5-base", learning_rate=5e-5):
        super().__init__()
        self.model = T5ForConditionalGeneration.from_pretrained(model_name)
        self.tokenizer = T5Tokenizer.from_pretrained(model_name)
        self.learning_rate = learning_rate

    def forward(self, input_ids, attention_mask, labels=None, past_key_values=None):
        # Use EncoderDecoderCache for past_key_values
        cache = None
        if past_key_values is not None:
            cache = EncoderDecoderCache.from_legacy_cache(past_key_values)
        return self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
            past_key_values=cache
        )

    def training_step(self, batch, batch_idx):
        # Indexing task
        index_outputs = self(
            input_ids=batch['input_ids'],
            attention_mask=batch['attention_mask'],
            labels=batch['docid_labels']
        )
        index_loss = index_outputs.loss

        # Retrieval task
        retrieval_outputs = self(
            input_ids=batch['input_ids'],
            attention_mask=batch['attention_mask'],
            labels=batch['retrieval_labels']
        )
        retrieval_loss = retrieval_outputs.loss

        # Combined loss
        loss = index_loss + retrieval_loss
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        index_outputs = self(
            input_ids=batch['input_ids'],
            attention_mask=batch['attention_mask'],
            labels=batch['docid_labels']
        )
        retrieval_outputs = self(
            input_ids=batch['input_ids'],
            attention_mask=batch['attention_mask'],
            labels=batch['retrieval_labels']
        )

        val_loss = index_outputs.loss + retrieval_outputs.loss
        self.log('val_loss', val_loss)
        return val_loss

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.learning_rate)

# Define the dataset class
class MSMARCODataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer, max_length=128, dense_model=None):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.dense_model = dense_model

    def generate_pseudo_queries(self, text):
        # Placeholder for pseudo-query generation
        return [f"Pseudo-query for: {text}"]

    def cluster_documents(self):
        # Placeholder for semantic clustering
        clustering = AgglomerativeClustering(n_clusters=10)
        return clustering.fit_predict([item['passage'] for item in self.data])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        inputs = self.tokenizer(
            item['query'],
            item['passage'],
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        )
        docid_labels = self.tokenizer(
            item['docid'],
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        )

        # Generate retrieval labels using dense model if available
        retrieval_labels = self.tokenizer(
            " ".join(self.generate_pseudo_queries(item['passage'])),
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            'input_ids': inputs['input_ids'].squeeze(0),
            'attention_mask': inputs['attention_mask'].squeeze(0),
            'docid_labels': docid_labels['input_ids'].squeeze(0),
            'retrieval_labels': retrieval_labels['input_ids'].squeeze(0)
        }

# Load the dataset and tokenizer
data = [  # Example dataset structure
    {'query': 'example query 1', 'passage': 'example passage 1', 'docid': '1'},
    {'query': 'example query 2', 'passage': 'example passage 2', 'docid': '2'}
]

val_data = [  # Example validation dataset
    {'query': 'example validation query 1', 'passage': 'example validation passage 1', 'docid': '3'},
    {'query': 'example validation query 2', 'passage': 'example validation passage 2', 'docid': '4'}
]

tokenizer = T5Tokenizer.from_pretrained("t5-base")
dataset = MSMARCODataset(data, tokenizer)
val_dataset = MSMARCODataset(val_data, tokenizer)

train_dataloader = DataLoader(dataset, batch_size=2)
val_dataloader = DataLoader(val_dataset, batch_size=2)

# Initialize the model
model = DSIT5Model()

# Define PyTorch Lightning Trainer
logger = WandbLogger(project="IR_DSI_Project")
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='checkpoints/',
    filename='dsi-t5-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,
    mode='min'
)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=3, mode='min')

trainer = pl.Trainer(
    max_epochs=5,
    logger=logger,
    callbacks=[checkpoint_callback, early_stopping_callback]
)

# Train the model
trainer.fit(model, train_dataloader, val_dataloaders=val_dataloader)


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


## 5: TRAINING

In [ ]:
# Model Checkpointing Callback.
checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",  # Metric to monitor
    dirpath="checkpoints/",  # Directory to save checkpoints
    filename="best-checkpoint-{epoch:02d}-{val_loss:.2f}",  # Checkpoint name format
    save_top_k=1,  # Save only the best model
    mode="min"  # Minimize the monitored metric
)

# Early Stopping Callback.
early_stopping_callback = EarlyStopping(
    monitor="val_loss",  # Metric to monitor
    patience=3,  # Number of epochs without improvement to wait
    mode="min"  # Minimize the monitored metric
)

# Learning Rate Monitoring Callback.
lr_monitor = LearningRateMonitor(logging_interval="step")

# StochasticWeightAveraging Callback.
swa_callback = StochasticWeightAveraging()


# Device Statistics Callback
device_stats_callback = DeviceStatsMonitor()

# Trainer implementation.
trainer = Trainer(
    max_epochs=3,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    devices=1 if torch.cuda.is_available() else None,
    enable_progress_bar=True,
    callbacks=[checkpoint_callback, early_stopping_callback, lr_monitor, swa_callback, device_stats_callback],
    gradient_clip_val=1.0,  # Clip gradients to this value
    precision=16,  # Enable 16-bit precision (AMP, Automatic Mixed Precision. Speed-Up Training and reduce Memory Usage)
)

In [ ]:
# Generate a timestamp for the run name
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Initialize WandB logger with the timestamp as the run name
wandb_logger = WandbLogger(
    project="IR_DSI",         # Shared project name
    name=f"run_{current_time}",     # Unique name based on the current time
    log_model=True                  # Log model artifacts
)

In [ ]:
# Initialize the model
model = MSMarcoClassifier()

# Initialize the Trainer.
trainer = Trainer(
    max_epochs=3,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    devices=1 if torch.cuda.is_available() else None,
    enable_progress_bar=True,
    logger=wandb_logger,
)
# Train the model.
trainer.fit(model, data_module)

## 6: TESTING